In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from collections import Counter
from sklearn.metrics import confusion_matrix

In [ ]:
help(tf.keras.datasets.fashion_mnist.load_data)

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
Counter(y_test)

In [ ]:
X_train = X_train / 255
X_test = X_test / 255

In [ ]:
def print_image_for_each_label(X, y):
    def find_index_of_label(y, label):
        for ix, item in enumerate(y):
            if item == label:
                return ix

    fig = plt.figure(figsize=(16, 6))
    labels = np.unique(y)
    for p, label in enumerate(labels):
        ix = find_index_of_label(y, label)
        image = X[ix, :, :]
        ax = fig.add_subplot(2, 5, p+1)
        plt.imshow(image, cmap=plt.cm.binary)
        # plt.imshow(image, cmap='gray')
        ax.set_title(label)
    plt.show()

In [ ]:
print_image_for_each_label(X_train, y_train)

In [ ]:
m_train, *image_size = X_train.shape
m_test = X_test.shape[0]

In [ ]:
X_train_flattened = X_train.reshape(m_train, -1)
X_test_flattened = X_test.reshape(m_test, -1)

In [ ]:
X_train_flattened.shape

### Image classification with logistic regression using scikit-learn

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(penalty='l2', C=0.05, random_state=2112, solver="newton-cg", multi_class="multinomial")

In [ ]:
model.fit(X_train_flattened, y_train)

In [ ]:
y_hat = model.predict(X_test_flattened)

In [ ]:
print(y_hat[:10])
print(y_test[:10])

In [ ]:
acc = model.score(X_test_flattened, y_test)

In [ ]:
print(acc)

In [ ]:
plt.figure(figsize=(7, 6))
plt.title('Confusion matrix', fontsize=14)
plt.imshow(confusion_matrix(y_test, y_hat))
plt.xticks(np.arange(10), list(range(10)), fontsize=12)
plt.yticks(np.arange(10), list(range(10)), fontsize=12)
plt.colorbar()
plt.show()
print("Test accuracy:", acc)

In [ ]:
well_classified = [ix for ix, (pred, ground) in enumerate(zip(y_hat, y_test)) if pred == ground]
misclassified = [ix for ix, (pred, ground) in enumerate(zip(y_hat, y_test)) if pred != ground]

In [ ]:
def show_image(matrix):
    fig = plt.figure(figsize=(4, 4))
    fig.add_subplot(1, 1, 1)
    plt.imshow(matrix, cmap=plt.cm.binary)
    plt.show()

In [ ]:
ix = well_classified[0]
show_image(X_test[ix])
print("label: {}".format(y_test[ix]))
print("prediction: {}".format(y_hat[ix]))

In [ ]:
ix = misclassified[0]
show_image(X_test[ix])
print("label: {}".format(y_test[ix]))
print("prediction: {}".format(y_hat[ix]))

### Image classification with a dense fully-connected Neural Network

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=50, verbose=1, validation_data=(X_test, y_test))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = np.arange(len(acc))

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')
plt.show()

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')
plt.show()

In [ ]:
y_hat = np.argmax(model.predict(X_test), axis=1)

In [ ]:
plt.figure(figsize=(7, 6))
plt.title('Confusion matrix', fontsize=14)
plt.imshow(confusion_matrix(y_test, y_hat))
plt.xticks(np.arange(10), list(range(10)), fontsize=12)
plt.yticks(np.arange(10), list(range(10)), fontsize=12)
plt.colorbar()
plt.show()
print("Test accuracy:", np.equal(y_hat, y_test).sum() / len(y_test))

### Experiment

Shift digits on a larger image randomly

In [ ]:
X_train.shape

In [ ]:
show_image(X_train[1])

In [ ]:
def recreate_images(X):
    nr_images = len(X)
    shifted_images = np.zeros(shape=(nr_images, 42, 42), dtype=X.dtype)
    for ix, image in enumerate(X):
        shifted = np.zeros((42, 42), dtype=image.dtype)
        row_ix, col_ix = np.random.randint(0, 14, size=2)
        shifted[row_ix:row_ix+28, col_ix:col_ix+28] = image
        shifted_images[ix, :, :] = shifted
    return shifted_images

In [ ]:
Z_train = recreate_images(X_train)
Z_test = recreate_images(X_test)

In [ ]:
print_image_for_each_label(Z_train, y_train)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(42, 42)),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(Z_train, y_train, epochs=50, verbose=1, validation_data=(Z_test, y_test))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = np.arange(len(acc))

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')
plt.show()

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')
plt.show()

In [ ]:
y_hat = np.argmax(model.predict(Z_test), axis=1)

In [ ]:
plt.figure(figsize=(7, 6))
plt.title('Confusion matrix', fontsize=14)
plt.imshow(confusion_matrix(y_test, y_hat))
plt.xticks(np.arange(10), list(range(10)), fontsize=12)
plt.yticks(np.arange(10), list(range(10)), fontsize=12)
plt.colorbar()
plt.show()
print("Test accuracy:", np.equal(y_hat, y_test).sum() / len(y_test))